<a href="https://colab.research.google.com/github/esmacicek/esmacicek/blob/main/vgg_egitim5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Gerekli kütüphaneleri yükleyin
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
# Veri yolu
train_path = '/content/drive/MyDrive/alz.dmn.project/Split/Train'
valid_path = '/content/drive/MyDrive/alz.dmn.project/Split/Validation'
test_path = '/content/drive/MyDrive/alz.dmn.project/Split/Test'

# Veri artırımı konfigürasyonu
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1./255
)

test_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1./255
)

# Veri setini yükleyin ve artırın
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    valid_path,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

# VGG16 modelini kullanın
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Yeni bir model oluşturun ve VGG16 modelini ekleyin
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

# Modeli derleyin
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

# Modeli eğitin
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# Modeli değerlendirin
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

accuracy = model.evaluate(test_generator)
print('Test Accuracy:', accuracy[1])

model.save('/content/drive/MyDrive/alz.dmn.project/Model/vgg.model.son.h5')

Found 6796 images belonging to 4 classes.
Found 2266 images belonging to 4 classes.
Epoch 1/50
106/106 [==============================] - 376s 4s/step - loss: 1.3921 - accuracy: 0.2995 - val_loss: 1.3469 - val_accuracy: 0.3585
Epoch 2/50
106/106 [==============================] - 116s 1s/step - loss: 1.3360 - accuracy: 0.3460 - val_loss: 1.3684 - val_accuracy: 0.3205
Epoch 3/50
106/106 [==============================] - 114s 1s/step - loss: 1.3475 - accuracy: 0.3394 - val_loss: 1.2887 - val_accuracy: 0.3750
Epoch 4/50
106/106 [==============================] - 115s 1s/step - loss: 1.2838 - accuracy: 0.3770 - val_loss: 1.2755 - val_accuracy: 0.3826
Epoch 5/50
106/106 [==============================] - 115s 1s/step - loss: 1.2292 - accuracy: 0.4078 - val_loss: 1.1631 - val_accuracy: 0.4339
Epoch 6/50
106/106 [==============================] - 115s 1s/step - loss: 1.1626 - accuracy: 0.4367 - val_loss: 1.0877 - val_accuracy: 0.4616
Epoch 7/50
106/106 [==============================] - 115s

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
